# Using Templates

Using [Jinja2](https://jinja.palletsprojects.com/en/3.0.x/) as template system. 

In [1]:
import jinja2 as J2
environment = J2.Environment()
template = environment.from_string("Hi {{ name }}, how are you?")
template.render(name="John")

'Hi John, how are you?'

A template system lets you replace a varible in a text. In fact, you can replace as many as variables as you want in all kinds of text. In all cases, the procedure is the same:

1. Create a template file. We will mark the file as template by using the extension `.j2`.
1. Get a list of values that you will use as input.
1. Write a script that will tie everything together.

For example, imagine you have a list of `hostname` and `IP` addresses, and want to create a configuration file for each host. The list of hosts is (see `hosts.txt`):

```
atta, 192.168.56.10
flik, 192.168.56.11
hopper, 192.168.56.12
```

And for each host create a configuration file for the Icinga monitoring system

```
/**
* Simple Icinga host configuration file
*/

object Host "atta" {
    check_command = "hostalive"
    address = " 192.168.56.10"
}
```

## Template

To use Jinja2, you create a template file, like the one in the `temps` folder:

```
/**
* Simple Icinga host configuration file
*/

object Host "{{ hostname }}" {
    check_command = "hostalive"
    address = "{{ ip_addr }}"
}
```

Where the place holders are marked with `{{ }}`, like `{{ hostname }}`

## Script

After creating a template, the next is to create a script that will read a input file, like `hosts.txt`, and apply the templating system

In [2]:
def mk_hosts(hosts_file):
    """
    Create Icinga configuration files from a list of hosts and IP addresses.
    """
    environment = J2.Environment(loader=J2.FileSystemLoader("temps/"))
    template = environment.get_template("hostconfig.j2")

    # Read file with the hosts list
    with open(hosts_file, "r") as fin:
        text = fin.readlines()
    hosts = [line.strip().split(",") for line in text]

    for host in hosts:

        hostname = host[0]
        ip_addr = host[1]
        content = template.render(hostname=hostname, ip_addr=ip_addr)

        with open(f"{hostname}.cfg", "w") as fout:
            fout.write(content)
            print(f"Wrote {hostname}")


In [3]:
mk_hosts('in/hosts.txt')

Wrote atta
Wrote flik
Wrote hopper
